# End to end tutorial for using the new Argilla SDK

This tutorial encomposses the full workflow of using the Argilla SDK to manage human feedback tasks. The main three areas are:

1. **Setting up a project**:
   - **Connecting to an Argilla server**:
   - **Creating a workspace**:
   - **Creating users**:
2. **Creating a dataset**:
   - **Defining the dataset's feedback task**:
   - **Adding records to the dataset**:
3. **Manipulating records**:
   - **Adding records with suggestions**
   - **Adding or Updating records with responses**

We will work through each of these areas and show three depths of control: A line, B line, and C line. The A line is the simplest and most abstracted, the B line is a bit more detailed, and the C line is the most detailed and flexible.


In [1]:
import argilla_sdk as rg

# 1. Setting up a project

- Connecting to an Argilla server
- Creating a workspace
- Creating users


In [2]:
client = rg.Argilla(api_url="http://localhost:6900", api_key="owner.apikey")

In [3]:
client.workspaces

datasets,name,id,updated_at
749,admin,47a2ace0-a9b2-4dde-9710-0656646a3de3,2024-01-31T18:48:50.856584
1,new_ws,b4b7410c-7014-4ae4-a923-ec095d4a7272,2024-03-19T05:59:06.362275
0,end-to-end,e5bf1534-b89a-4245-9a30-99284710a1a9,2024-04-08T13:41:11.967896


In [4]:
ws = rg.Workspace("end-to-end", client=client)

In [5]:
try:
    ws.create()
except Exception as e:
    print("Workspace exists")

Workspace exists


In [6]:
user = rg.User(username="end-user", password="password", client=client)

try:
    ws.create()
except Exception as e:
    print("User exists")

User exists


# 2. Creating a dataset

- Defining the dataset's feedback task
- Adding records to the dataset
- Collecting the records

In [7]:
# !pip install -qqq datasets

from uuid import uuid4

from datasets import load_dataset


ds = load_dataset("imdb", split="train[:100]").to_list()

ds = [{"text": x["text"], "label": "positive" if x["label"] == 1 else "negative", "id": uuid4()} for x in ds]

print(ds[:5], sep="\n")

/home/ben/code/argilla-python/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far b

In [8]:
# Defining the dataset's feedback task
settings = rg.Settings(
    guidelines="Classify the articles into one of the four categories.",
    fields=[
        rg.TextField(
            name="text",
            title="Text from the article",
            use_markdown=False,
        ),
    ],
    questions=[
        rg.LabelQuestion(
            name="label",
            title="In which category does this article fit?",
            labels=["positive", "negative"],
        )
    ],
)

In [9]:
dataset = rg.Dataset(
    name=f"IMDB-dataset-{uuid4()}",
    workspace_id=str(client.workspaces[0].id),
    settings=settings,
    client=client,
)

# 3. Manipulating records

## Adding records with suggestions

Argilla handles labels in the dataset with the same name as the question as suggestions. For example, if you have a `rg.TextQuestion(name="label")` and a `label` key in your sample, Argilla will assign the values of `label` to the question `label`. Below we'll customize this behaviour.

In [10]:
dataset.create()
dataset.records.add(records=ds)

/home/ben/code/argilla-python/src/argilla_sdk/records/_resource.py:192: UserWarning: Record attribute external_id is not in the schema so skipping. 
                    Define a mapping to map source data fields to Argilla Fields, Questions, and ids
                    
  warnings.warn(


We can now pull and export the records from Argilla and see that we have dictionaries with a key `label.suggestion` that reflects the source data.

In [11]:
records = dataset.records.to_list()
records[:5]

[{'id': UUID('4f29315f-67cf-4650-a887-d86abe9bf8dd'),
  'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographi

## Adding or updating records with reponses

Let's say that the source data contains responses that we want to handle as true values, for example mannually annotated labels. We can use a  `mapping` to map the keys in our source data to Argilla questions, and specify `response`. 

In [12]:
dataset.records.update(records=ds, mapping={"label": "label.response"})

            No existing records founds to update. 
            If you want to add new records, you should use the `Dataset.records.add` method.
            


In [13]:
records = dataset.records.to_list()
records[:5]

[{'id': UUID('5f604b00-e2b2-4c54-9a67-c563a1a09d05'),
  'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographi

In this way, we could also handle diverse naming in our source data, so we could deal with:


In [16]:
source_data = [{"x": "I'm a data sample in a dataset with vague field names :(", "y": "negative", "idx": "90146"}]
dataset.records.add(records=source_data, mapping={"y": "label", "x": "text", "idx": "id"})